In [1]:
import os
from pathlib import Path

# Geospatial
import geopandas as gpd
import fiona
from rtree.index import Index

In [2]:
# Enable KML drivers for fiona
fiona.drvsupport.supported_drivers['kml'] = 'rw'
fiona.drvsupport.supported_drivers['KML'] = 'rw'

In [3]:
# Set path to directory
dir_path = os.path.join(str(Path.home()), 'Desktop', "forest_mon", "data")

# Path to AoI
aoi_path = os.path.join(dir_path, "dream_gardens.kml")

# Path to WRS shapefile
wrs_path = os.path.join(dir_path, "WRS2_descending.shp")

# Path to Sentinel-2 tiles
s2_path = os.path.join(dir_path, "S2_tiles.kml")

In [4]:
# Create WRS gdf
wrs = gpd.GeoDataFrame.from_file(wrs_path)

# Create Sentinel-2 gdf
s2 = gpd.GeoDataFrame.from_file(s2_path)

# Create AoI gdf
aoi = gpd.GeoDataFrame.from_file(aoi_path)

In [5]:
def fast_intersect(aoi, grid):
    """
    Uses spatial indexing to find tiles that intersect 
    the area of interest
    
    Params
    ------
    
    aoi: Geopandas GeoDataFrame
        GeoDataFrame containing area of interest for
        which you want to find an intersection
        
    grid: Geopandas GeoDataFrame
        This will intersect the AOI
        
    Returns
    -------
    precise_matches: Geopandas GeoDataFrame
        This is the resulting gdf that contains grid features
        that intersect area of interest
    """
    
    # Create spatial index for gdf
    grid_spatial_idx = grid.sindex
    
    # Convert AoI coords into something gpd can understand
    aoi_boundaries = (aoi.bounds.minx[0],
                      aoi.bounds.miny[0],
                      aoi.bounds.maxx[0],
                      aoi.bounds.maxy[0])
    
    # Get possible matches using rtree, return as list
    poss_matches_idx = list(grid_spatial_idx.intersection(aoi_boundaries))
    
    # Get possible matches rows from gdb
    poss_matches = grid.iloc[poss_matches_idx]
    
    # Get precise matches by selecting from possible matches
    # where they intersect AoI
    precise_matches = poss_matches[poss_matches.intersects(aoi.geometry[0])]

    return precise_matches

In [8]:
fast_intersect(aoi=aoi, grid=wrs)

,AREA,PERIMETER,PR_,PR_ID,RINGS_OK,RINGS_NOK,PATH,ROW,MODE,SEQUENCE,WRSPR,PR,ACQDayL7,ACQDayL8,geometry
17273,3.0175,7.01407,2022.0,2022.0,1,0,149,38,D,33270,149038,149038,10,2,"POLYGON ((72.85290 31.12687, 72.85270 31.12690..."


In [9]:
fast_intersect(aoi=aoi, grid=s2)

,Name,Description,geometry
40469,43RDQ,TILE PROPERTIES<br><table border=0 cellpadding...,GEOMETRYCOLLECTION Z (POLYGON Z ((73.94510 31....
